In [1]:
#import libraries and data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.metrics import classification_report_imbalanced
from imblearn.under_sampling import ClusterCentroids


mitbih_test = pd.read_csv('../../input/mitbih_test.csv')
mitbih_train = pd.read_csv('../../input/mitbih_train.csv')
ptbdb_abnormal = pd.read_csv('../../input/ptbdb_abnormal.csv')
ptbdb_normal = pd.read_csv('../../input/ptbdb_normal.csv')

#rename columns

for df in [ptbdb_abnormal, ptbdb_normal, mitbih_test, mitbih_train]:
    df.columns = [i for i in range(len(df.columns))]

#combine datasets, remove class 4, combine classes 1,2,3

ptbdb = pd.concat([ptbdb_abnormal, ptbdb_normal])
mitbih = pd.concat([mitbih_train, mitbih_test])

mitbih_recoded = mitbih.loc[mitbih[187] != 4]
mitbih_recoded.loc[:, 187] = mitbih_recoded[187].replace([1,2,3], 1)

df_total = pd.concat([mitbih_recoded, ptbdb])

#split into train and test

X = df_total.drop(187, axis=1)
y = df_total[187]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [2]:
#create lr model and param grid

lr = LogisticRegression()
param_grid = {
    'max_iter': [100, 500, 1000, 5000],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': [None, 'l2']
    }

#create grid search, fit to data, evaluate

grid = GridSearchCV(lr, param_grid, scoring='accuracy', cv=5)
grid.fit(X_train, y_train)

print('best params: ', grid.best_params_)
print('best score: ', grid.best_score_)

c:\Users\maxjj\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\maxjj\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\maxjj\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_rati

best params:  {'C': 100, 'max_iter': 500, 'penalty': 'l2'}
best score:  0.8391401850117306


In [3]:
#evaluate best estimator on training and test set

lr_best = grid.best_estimator_

print('best estimator score on training set: ', lr_best.score(X_train, y_train))
print('best estimator score on test set: ', lr_best.score(X_test, y_test))
print('best estimator classification report: \n', classification_report(y_test, lr_best.predict(X_test)))

best estimator score on training set:  0.839948686990643
best estimator score on test set:  0.8410159113449183
best estimator classification report: 
               precision    recall  f1-score   support

         0.0       0.85      0.97      0.91     18923
         1.0       0.69      0.25      0.36      4268

    accuracy                           0.84     23191
   macro avg       0.77      0.61      0.64     23191
weighted avg       0.82      0.84      0.81     23191

